# Enfriamiento Simulado

In [1]:
import pandas as pd
import numpy as np
import timeit
from scripts.queso_model import model_data, model_vars, objective_func, balance, alloc_df

# Escenario 1
folder = './data/escenario_1/' # Carpeta con los datos
demanda = 60 # Demanda del cliente
ctiempo = 1000 # Costo por unidad de tiempo

#Escenario 2
#folder = './data/escenario_2/' # Carpeta con los datos
#demanda = 12000 # Demanda del cliente
#ctiempo = 1000 # Costo por unidad de tiempo

# Escenario 3
#folder = './data/escenario_3/' # Carpeta con los datos
#demanda = 100 # Demanda del cliente
#ctiempo = 1000 # Costo por unidad de tiempo

# Escenario 4
#folder = './data/escenario_4/' # Carpeta con los datos
#demanda = 20000 # Demanda del cliente
#ctiempo = 1000 # Costo por unidad de tiempo


info_acopios = 'centros_acopio.xlsx'
costo_transporte = 'costos_transporte.xlsx'
tiempo_transporte = 'tiempos_transporte.xlsx'

archivos = {
    'info_acopios': info_acopios,
    'costo_transporte': costo_transporte,
    'tiempo_transporte': tiempo_transporte,
}

data = model_data(archivos, demanda, ctiempo, folder=folder)

demanda = data['demanda']
ctiempo = data['ctiempo']
params_df = data['params_df']

N, seed, capacidades = model_vars(params_df)

def gen_s0(n_vars):
    gen_vector = np.zeros(n_vars+1, dtype=float)
    indices = np.arange(n_vars)
    np.random.shuffle(indices)
    while np.sum(gen_vector) < demanda and indices.size > 0:
        idx = indices[0]
        gen_vector[idx] += capacidades[idx]
        indices = np.delete(indices, 0)
        if np.sum(gen_vector) > demanda:
            gen_vector[idx] = gen_vector[idx] - (np.sum(gen_vector) - demanda)
            break
    gen_vector[n_vars] = np.random.randint(capacidades[n_vars] + 1)
    return gen_vector

def gen_s(sol):
    indices = np.arange(N*2+1)
    np.random.shuffle(indices)
    idx = indices[0]
    while indices.size > 0:
        if idx == N*2:
            cap = sol[idx].astype(int)
            cap = np.delete(np.arange(capacidades[N*2]+1), cap)
            sol[idx] = np.random.choice(cap)
            break
        if sol[idx] == 0 == capacidades[idx]:
            indices = np.delete(indices, 0)
            idx = indices[0]
            continue
        s = np.delete(sol, N*2)
        if s[idx] == 0:
            delta = capacidades[idx]
            s[idx] = capacidades[idx]
            diff = True
        else:
            delta = s[idx]
            s[idx] = 0
            diff = False
        balance(s, capacidades, delta, diff)
        sol = np.append(s, sol[N*2])
        break
    return sol

def anneal(t, t_min, e_th, alpha):
    f = objective_func
    s = gen_s0(N*2)
    e = f(s, N, data)
    k = 0
    historial_f = []
    while t > t_min and e > e_th:
        s_new = gen_s(s)
        e_new = f(s_new, N, data)
        historial_f.append(e)
        delta = e_new - e
        if delta < 0:
            s = s_new
            e = e_new
        else:
            p = np.exp(-delta / t)
            r = np.random.rand()
            if r < p:
                s = s_new
                e = e_new
        t *= alpha
        k += 1
    return s, k, historial_f

def sa(t_max, t_min, e_th, alpha, key=1):
    x, count, historial_f = anneal(t_max, t_min, e_th, alpha)
    return x, historial_f

def run_experiment(t_inicial, alpha, block_name, runs=100):
    results = []
    for run in range(runs):
        t_start = timeit.default_timer()
        annealing, historial = sa(t_inicial, 1e-5, 1.5e5, alpha)
        t_end = timeit.default_timer()
        t_delta = t_end - t_start
        costo = objective_func(annealing, N, data)
        results.append({
            'block': block_name,
            'T_inicial': t_inicial,
            'alpha': alpha,
            'run': run + 1,
            'costo': costo,
            'tiempo': t_delta
        })
    return results 

def summarize_experiment(results):
    df = pd.DataFrame(results)
    summary = df.groupby(['block', 'T_inicial', 'alpha']).agg(
        mean=('costo', 'mean'),
        std=('costo', 'std'),
        avg_time=('tiempo', 'mean')
    ).reset_index()
    summary['var.coeff'] = summary['std'] / summary['mean']
    summary = summary[['block', 'T_inicial', 'alpha', 'mean', 'var.coeff', 'avg_time']]
    return summary

block1_t_inicial = [50, 75, 100]
block1_alpha = [0.5, 0.7, 0.9]
block2_t_inicial = [100, 200, 300]
block2_alpha = np.arange(0.900, 1.000, 0.011).tolist()

all_results = []
for t_inicial in block1_t_inicial:
    for alpha in block1_alpha:
        all_results.extend(run_experiment(t_inicial, alpha, 'Bloque 1'))

for t_inicial in block2_t_inicial:
    for alpha in block2_alpha:
        all_results.extend(run_experiment(t_inicial, alpha, 'Bloque 2'))

results_df = pd.DataFrame(all_results)
summary_df = summarize_experiment(all_results)

results_df.to_excel('experiment_results_sa.xlsx', index=False)
summary_df.to_excel('experiment_summary_sa.xlsx', index=False)